***WEB SCRAPING***

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

In [5]:
# TASK 1: Request the Falcon9 Launch Wiki page from its URL

response = requests.get(static_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print("Page retrieved successfully!")
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Page retrieved successfully!


In [6]:
# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, "html.parser")

# Just to verify, print the page title
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [7]:
# Verify BeautifulSoup object
print(soup.title)
print(soup.title.string)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [8]:
# TASK 2: Extract all column/variable names from the HTML table header

# Find all tables in the page
html_tables = soup.find_all("table")

# Print how many tables were found
print(f"Total tables found: {len(html_tables)}")

# Just to preview the first one
print(html_tables[0].prettify()[:500])


Total tables found: 25
<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <div class="mw-heading mw-heading3">
     <h3 id="Rocket_configurations">
      Rocket configurations
     </h3>
    </div>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
     <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
      <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-lef


In [9]:
# Select the third table (index 2) which contains launch records
first_launch_table = html_tables[2]

# Print out the full table HTML (may be long)
print(first_launch_table.prettify()[:1000])  # show first 1000 characters for readability


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

In [10]:
# TASK 2 continued: Extract column names

column_names = []

# Find all <th> elements inside the first launch table
table_headers = first_launch_table.find_all("th")

# Iterate over each <th> element
for th in table_headers:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Print the extracted column names
print(column_names)


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [13]:
# Initialize launch_dict with empty lists
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

extracted_row = 0

# Extract each table with class 'wikitable plainrowheaders collapsible'
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # check to see if first table heading is a number (Flight number)
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all('td')

        if flag:
            extracted_row += 1

            # Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not(bv) and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else row[5].text.strip()
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else row[6].text.strip()
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

print(f"Total launch records extracted: {extracted_row}")

# Convert dictionary into DataFrame
df = pd.DataFrame({ key: pd.Series(value) for key, value in launch_dict.items() })

# Show first 5 rows
print(df.head())


Total launch records extracted: 121
  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18    No attempt\n   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18    No attempt\n   

              Date   Time  
0      4 June 2010  18:45  
1  8 December 2010

In [14]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })